# Scrape PTA Trunojoyo
> Website : https://pta.trunojoyo.ac.id <br>

## **Web Scraping**
> **Web Scraping** adalah metode yang digunakan untuk mengumpulkan informasi, baik angka, teks, tabel, maupun media dari berbagai website menggunakan sebuah software yang dapat menyimpan dan mengatur informasi yang telah diunduh.

In [1]:
_= !pip install bs4

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
class ScrapingPtaTrunojoyo:
  def __init__(self, url):
    self.url = url
  
  def page(self):
    res = requests.get(self.url)
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find('ol', class_='pagination').findChildren(recursive=False)
    return int(contents[-1].a['href'].split('/')[-1])

  def artikel_page(self, page):
    res = requests.get(f'{self.url}/{page}')
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find_all('li', attrs={'data-cat':'#luxury'})
    return contents

  def get_berita(self):
    pta_list = []
    page = self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in artikels:
        link = artikel.find('a', class_='gray button')['href']
        respone = requests.get(link)
        soup_data = BeautifulSoup(respone.content, 'html.parser')
        cont = soup_data.find('div', attrs={'id':'content_journal'})
        # ambil data
        nim = link[-12:]
        nama = cont.findAll('div')[1].div.span.text.split(' : ')[1]
        judul = ' '.join(cont.find('a', class_='title').get_text().splitlines()).capitalize()
        abstrak = ' '.join(cont.find('p').get_text().splitlines()).capitalize()
        if abstrak == '':
          abstrak = ' '.join(cont.find('p').findNext('p').stripped_strings).capitalize()
        # judul = str.join("", (cont.find('a', class_='title').string).splitlines()).capitalize()
        # abstrak = str.join("", (cont.find('p').string).splitlines()).capitalize()
        pta_list.append({
            'NPM' : nim,
            'Penulis' : nama,
            'Judul': judul,
            'Abstrak': abstrak
        })
    self.pta = pta_list
  
  def result(self):
    return pd.DataFrame(self.pta)
  
  def save(self, namefile='pta'):
    df = pd.DataFrame(self.pta).to_csv(namefile+'.csv', index=False)
    print('Berhasil di save')

In [4]:
scPta = ScrapingPtaTrunojoyo('https://pta.trunojoyo.ac.id/c_search/byprod/10/')

In [5]:
scPta.get_berita()

In [6]:
scPta.result()

,NPM,Penulis,Judul,Abstrak
0,040411100468,A.Ubaidillah S.Kom,Perancangan dan implementasi sistem database ...,Sistem informasi akademik (siakad) merupaka...
1,040411100476,"M. Basith Ardianto,",Aplikasi kontrol dan monitoring jaringan kompu...,Berjalannya koneksi jaringan komputer dengan l...
2,040411100480,"Akhmad Suyandi, S.Kom",Rancang bangun aplikasi proxy server untuk enk...,Web server adalah sebuah perangkat lunak serve...
3,070411100070,Heri Supriyanto,Sistem pendukung keputusan optimasi penjadwala...,Penjadwalan kuliah di perguruan tinggi me...
4,080411100115,Septian Rahman Hakim,Sistem augmented reality animasi benda bergera...,Seiring perkembangan teknologi yang ada diduni...
...,...,...,...,...
853,160411100032,Rachmad Agung Pambudi,Penerapan algoritma long-short term memory unt...,Investasi saham selama ini memiliki resiko ker...
854,160411100182,Nadila Hidayanti,Sistem pencarian teks al-quran terjemahan berb...,Information retrieval (ir) merupakan pengambil...
855,160411100077,Afni Sakinah,Klasifikasi kompleksitas visual citra sampah m...,Klasifikasi citra merupakan proses pengelompok...
856,160411100084,Friska Fatmawatiningrum,Identifikasi biner atribut pejalan kaki menggu...,Identifikasi atribut pejalan kaki merupakan sa...
